In [1]:
%load_ext autoreload
%autoreload 2
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/mnt/d/1ResProj/mob2crime'

In [2]:
import pandas as pd
import numpy as np
import geopandas as gp
import folium
from scipy.spatial import Voronoi, voronoi_plot_2d

from src.utils.gis import *
from src.utils.map_vis import *
import src.mex_helper as mex
%matplotlib inline

In [3]:
from shapely.ops import cascaded_union 

In [4]:
tvor = mex.tower_vor()

loading existing tvor file: data/mex_tower/mex_tvor_in_country.geojson


In [5]:
population = pd.read_csv('data/mexico/Localidades-population.csv')
population['loc_id'] = population['Clave de localidad'].apply(lambda x: f'{x:09}')

In [6]:
population['CVE_ENT'] = population['Clave entidad'].apply(lambda x: f'{x:02}')

In [7]:
LUR = gp.read_file('data/mexico/inegi2018/Marco_Geoestadistico_Integrado_diciembre_2018/conjunto de datos/01_32_l.shp')
LUR['loc_id'] = LUR.CVE_ENT+LUR.CVE_MUN+LUR.CVE_LOC
LUR4326 = LUR.to_crs(epsg=4326)
LUR4326 = LUR4326.merge(population[['loc_id','Población total']], how='left')
LUR4326= LUR4326.rename(columns={'Población total':'Pop'})

In [8]:
LPR = gp.read_file('data/mexico/inegi2018/Marco_Geoestadistico_Integrado_diciembre_2018/conjunto de datos/01_32_lpr.shp')
LPR['loc_id'] = LPR.CVE_ENT+LPR.CVE_MUN+LPR.CVE_LOC
LPRBf = LPR.copy()
LPRBf.geometry = LPRBf.buffer(500)
LPRBf4326 = LPRBf.to_crs(epsg=4326)
LPRBf4326 = LPRBf4326.merge(population[['loc_id','Población total']], how='left')
LPRBf4326= LPRBf4326.rename(columns={'Población total':'Pop'})

In [9]:
LPRBf4326_Ponly = LPRBf4326[~LPRBf4326.loc_id.isin(LUR4326.loc_id)]

In [10]:
# most localidad popluation has geographic information 
df = pd.DataFrame([
    LUR4326.groupby('CVE_ENT')['Pop'].sum(),
    LPRBf4326_Ponly.groupby('CVE_ENT')['Pop'].sum(), 
    population.groupby('CVE_ENT')['Población total'].sum()
]).T


df.columns=['pop_poly', 'pop_point', 'pop_all']
df['pop_shap'] = df.pop_poly+df.pop_point

df['poly_all'] = df.pop_poly/df.pop_all
df['shap_all'] = df.pop_shap/df.pop_all


df

,pop_poly,pop_point,pop_all,pop_shap,poly_all,shap_all
CVE_ENT,,,,,,
01,1160573.0,23382.0,1184996.0,1183955.0,0.979390,0.999122
02,3114410.0,35255.0,3155070.0,3149665.0,0.987113,0.998287
03,528390.0,18242.0,637043.0,546632.0,0.829442,0.858077
04,802510.0,19263.0,822491.0,821773.0,0.975707,0.999127
05,2723488.0,20189.0,2748395.0,2743677.0,0.990938,0.998283
06,641429.0,8421.0,650567.0,649850.0,0.985954,0.998898
07,4126322.0,614493.0,4799085.0,4740815.0,0.859814,0.987858
08,3245119.0,159972.0,3406481.0,3405091.0,0.952631,0.999592
09,8824741.0,24953.0,8851080.0,8849694.0,0.997024,0.999843


In [11]:
LUR_pop = LUR4326[LUR4326.Pop>0].drop(['CVEGEO','CVE_LOC'],axis=1)
LPR_pop = LPRBf4326_Ponly[LPRBf4326_Ponly.Pop>0].drop(['CVE_LOC', 'CVE_AGEB', 'CVE_MZA', 'PLANO', 'CVEGEO'],axis=1)
LPR_pop['AMBITO'] = 'Rural'

L_pop = pd.concat([LUR_pop, LPR_pop[LUR_pop.columns]], ignore_index=True).set_index('loc_id')

In [12]:
t2loc = polys2polys(tvor, L_pop, pname1='tower', pname2='localidad', cur_crs=4326, area_crs=mex.AREA_CRS, intersection_only=False)

In [13]:
loc_area = t2loc.groupby('localidad').iarea.sum()

In [14]:
loc_area.name='loclidad_area'

In [15]:
t2loc = t2loc.merge(L_pop[['Pop']], left_on='localidad', right_index=True)

In [16]:
t2loc = t2loc.drop(['localidad_area','weight'], axis=1).merge(loc_area, left_on='localidad', right_index=True)

In [17]:
t2loc['iPop'] = t2loc.Pop*t2loc.iarea/t2loc.loclidad_area

In [18]:
tower_cover_pop = t2loc.groupby('tower').iPop.sum()
tower_cover_pop.name='tower_pop'

In [19]:
t2loc = t2loc.merge(tower_cover_pop, left_on='tower', right_index=True)

In [20]:
t2loc['weight'] = t2loc.iPop/t2loc.tower_pop

In [21]:
t2loc.head()

,tower,localidad,geometry,iarea,tower_area,Pop,loclidad_area,iPop,tower_pop,weight
0,55F17E0F-55F17E10-55F17E11,220160028,"POLYGON ((-100.0470458903587 20.3503507702433,...",278138.154568,2.077944e+07,1682.0,624556.371848,749.057086,20461.254045,0.036609
3,55F17E0F-55F17E10-55F17E11,220160365,POLYGON ((-100.0291409133007 20.36072658763791...,258766.261898,2.077944e+07,8.0,784137.122572,2.640010,20461.254045,0.000129
5,55F17E0F-55F17E10-55F17E11,220160134,POLYGON ((-100.0231424730812 20.36566652065543...,63853.672927,2.077944e+07,5.0,784137.122250,0.407159,20461.254045,0.000020
7,55F17E0F-55F17E10-55F17E11,220160125,POLYGON ((-100.0189607303471 20.36911033742224...,27627.217029,2.077944e+07,1448.0,543757.985267,73.569881,20461.254045,0.003596
9,55F17E0F-55F17E10-55F17E11,220160112,POLYGON ((-100.0226304583079 20.37222202881638...,48976.285720,2.077944e+07,67.0,48976.285720,67.000000,20461.254045,0.003274


In [22]:
m = folium.Map(location=[19.381495, -99.139095], zoom_start=6)
geojson_per_row(tvor, 'Tower', some_map=m)
geojson_per_row(t2loc[t2loc.localidad.isin(population[population.CVE_ENT.isin(['01','02'])].loc_id)], 'T2loc', some_map=m, 
                color='green', tip_cols=['localidad','tower','weight', 'Pop','tower_pop', 'iPop'])
# geojson_per_row(target_area, 'poly', color='green', some_map=m)
# geojson_per_row(target_area2, 'circle', color='yellow', some_map=m)
folium.LayerControl().add_to(m)
# m.save('maps/20190829VorByPop.html')

NotImplementedError: Multi-part geometries do not provide a coordinate sequence